In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

scala_version = '2.12'
spark_version = '3.3.0'
# TODO: Ensure match above values match the correct versions
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.2.1'
]

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkProductCount") \
    .config("spark.jars.packages", ",".join(packages)) \
    .getOrCreate()

In [2]:
rawData = spark.readStream.format("socket").option("host", "localhost") \
.option("port", "9999") \
.option("includeTimestamp", True) \
.load()

In [3]:
# query = rawData.select((rawData.value).alias("product")
#                 , (rawData.timestamp).alias("time")) \
#         .groupBy(window("time", "1 minutes"), "product").count()

query = rawData.select((rawData.value).alias("product"),(rawData.timestamp).alias("time")).groupBy("product").count()

In [12]:
checkpointDir = "/tmp/spark-kafka/"

result = query.selectExpr("cast(product as string) as value").writeStream \
.outputMode("complete").format("kafka") \
.option("kafka.bootstrap.servers", "localhost:9092").option("topic", "productCounts")\
.option("checkpointLocation", checkpointDir) \
.start()

# query.awaitTermination()

In [13]:
result.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [70]:
result.stop()